 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P TSX 60 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp_tsx_history.csv`: Contains historical closing prices of the S&P TSX 60 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data.

In [ ]:
whale_returns_path = Path("./Resources/whale_returns.csv")

In [ ]:
# Reading whale returns
df_whale_returns = pd.read_csv(whale_returns_path, index_col = 'Date')

df_whale_returns.head()

In [ ]:
# Count nulls
df_whale_returns.isnull().sum()


In [ ]:
# Drop nulls
df_whale_returns = df_whale_returns.dropna()

df_whale_returns.index = pd.to_datetime(df_whale_returns.index)

df_whale_returns.sort_index(inplace=True)

df_whale_returns

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data.

In [ ]:
# Reading algorithmic returns
algo_returns_path = Path("./Resources/algo_returns.csv")

df_algo_returns = pd.read_csv(algo_returns_path, index_col = 'Date')

df_algo_returns.head()

In [ ]:
# Count nulls
df_algo_returns.isnull().sum()

In [ ]:
# Drop nulls
df_algo_returns = df_algo_returns.dropna()

df_algo_returns.index = pd.to_datetime(df_algo_returns.index)

df_algo_returns.sort_index(inplace=True)

df_algo_returns

## S&P TSX 60 Returns

Read the S&P TSX 60 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P TSX 60 Closing Prices
sp_tsx_path = Path("./Resources/sp_tsx_history.csv")

sp_tsx_returns = pd.read_csv(sp_tsx_path, index_col = 'Date')

sp_tsx_returns.head()

In [ ]:
# Check Data Types

sp_tsx_returns.index = pd.to_datetime(sp_tsx_returns.index)

sp_tsx_returns


In [ ]:
# Fix Data Types

sp_tsx_returns['Close']=sp_tsx_returns['Close'].str.replace('$','')

sp_tsx_returns['Close']=sp_tsx_returns['Close'].str.replace(',','')

In [ ]:
sp_tsx_returns['Close']= sp_tsx_returns['Close'].astype('float')

sp_tsx_returns

In [ ]:
# Calculate Daily Returns
sp_tsx_daily_returns = sp_tsx_returns.pct_change()

sp_tsx_daily_returns

In [ ]:
# Drop nulls
sp_tsx_daily_returns = sp_tsx_daily_returns.dropna()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sp_tsx_daily_returns.rename(columns={'Close':'S&P500'},inplace=True)

sp_tsx_daily_returns.sort_index(inplace = True)

sp_tsx_daily_returns

## Combine Whale, Algorithmic, and S&P TSX 60 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P TSX 60 Returns into a single DataFrame with columns for each portfolio's returns.
combined_daily_returns = pd.concat([df_whale_returns, df_algo_returns, sp_tsx_daily_returns], axis = 'columns', join = 'inner')
combined_daily_returns

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios

combined_daily_returns.plot(legend=True, figsize = (10,10), title='Portfolios Daily Returns')

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios

cumulative_returns = (1+combined_daily_returns).cumprod()

# Plot cumulative returns

cumulative_returns.plot(legend=True, figsize = (10,10), title='Portfolios Cummulative Returns')

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios.
4. Determine which portfolios are riskier than the S&P TSX 60.
5. Calculate the Annualized Standard Deviation.

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
combined_daily_returns.plot(kind = 'box',figsize=(20,10), title='Portfolios Daily Returns')

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
combined_daily_std = combined_daily_returns.std()

combined_daily_std

### Determine which portfolios are riskier than the S&P TSX 60

In [ ]:
# Calculate  the daily standard deviation of S&P TSX 60
combined_daily_returns['S&P500'].std()

portfolios_list = combined_daily_returns.columns


# Determine which portfolios are riskier than the S&P TSX 60
riskier_portfolios = []

for portfolio in portfolios_list:
    if combined_daily_returns[portfolio].std() > combined_daily_returns['S&P500'].std():
        riskier_portfolios.append(portfolio)
        
riskier_portfolios        


### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
combined_annual_std = combined_daily_std * np.sqrt(252)

combined_annual_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window.
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P TSX 60.
3. Choose one portfolio, then calculate and plot the 60-day rolling beta for it and the S&P TSX 60.

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window

rolling_std = combined_daily_returns.rolling(window=21).std()

rolling_std

# Plot the rolling standard deviation

rolling_std.plot(figsize = (15,10), title='21-Day Rolling Standard Deviation')


### Calculate and plot the correlation

In [ ]:
# Calculate the correlation

portfolio_corr = combined_daily_returns.corr()

# Display de correlation matrix

portfolio_corr


### Calculate and Plot Beta for a chosen portfolio and the S&P 60 TSX

In [ ]:
# Calculating Beta of Portfolios and the S&P 500

sp500_rolling_var = combined_daily_returns['S&P500'].rolling(window=60).var() # Calculate variance of S&P TSX

# Using Functionns to Calculate Rolling Beta of Selected Portfolio

def calculate_rolling_beta():
    rolling_beta = pd.DataFrame()
    for portfolio in portfolios_list:
        portfolio_rolling_cov = combined_daily_returns[portfolio].rolling(window=60).cov(combined_daily_returns['S&P500']) # Calculate covariance of a single portfolio
        portfolio_rolling_beta = portfolio_rolling_cov / sp500_rolling_var                                                 # Computing beta
        rolling_beta[portfolio] = portfolio_rolling_beta
    
    return rolling_beta.dropna()

portfolios_rolling_beta = calculate_rolling_beta()

portfolios_rolling_beta

# Plot beta trend for selected portfolio from portfolios list

portfolios_rolling_beta[portfolios_list[4]].plot(figsize = (18,6), title=(f'{portfolios_list[4]} 60-Day Rolling Beta Trend'))
                                            
        

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window

portfolio_std_ewm = combined_daily_returns.ewm(com=21).std()

portfolio_std_ewm

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios

portfolio_sharpe = (combined_daily_returns.mean() * 252)/((combined_daily_returns.std())*np.sqrt(252))

portfolio_sharpe

In [ ]:
# Visualize the sharpe ratios as a bar plot
portfolio_sharpe.plot(kind='bar', figsize=(10,5), title='Portfolio Sharpe Ratios')

### Determine whether the algorithmic strategies outperform both the market (S&P TSX 60) and the whales portfolios.

While Algo 1 portfolio clearly outperformed the other portfolios and the market comprehensively, Algo 2 had better reward for risk than market and other whale portfolios except for Berkshire Hathaway Inc.

With low Beta and Sharpe ratio of 1.50, portfolio constructed using Algo 1 strategy has been a clear winner giving a higher risk-adjusted return with lesser volatility compared to other portofolios.

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P TSX 60. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock.
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns.
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others.
5. Include correlation analysis to determine which stocks (if any) are correlated.

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Using GOOGLEFINANCE to generate Stock Price Data 
# GOOGLEFINANCE("NYSE:CVX", "price", DATE(2018,2,1), DATE(2019,12,31), "DAILY")
# Stocks selected for analysis NVDA, SHOP, BAC, CVX, OTEX

stocks_list = ['NVDA','SHOP','BAC','CVX','OTEX']

# Using Functions to read stocks data and create DataFrame of Daily Returns

def daily_returns():
    dfs = []
    for stock in stocks_list:
        file_path = Path("./Resources/"+stock+".csv")
        stock_df = pd.read_csv(file_path, index_col = 'Date') # Reading data from stocks
        stock_df.rename(columns = {'Close':(f'{stock}')}, inplace = True)
        stock_df.index = pd.to_datetime(stock_df.index).date # Reset Date index
        stock_df.sort_index(inplace=True)
        dfs.append(stock_df)
        
    stocks_daily_price = pd.concat(dfs, axis = 'columns', join = 'inner') # Combine all stocks in a single DataFrame
    
    stocks_daily_price.drop(columns = 'Symbol', inplace = True) # Reorganize portfolio data by having a column per symbol
    
    stocks_daily_returns = stocks_daily_price.pct_change() # Calculate daily returns
    
    stocks_daily_returns.dropna(inplace = True) # Drop NAs
    
    return stocks_daily_returns

my_stocks_daily_returns = daily_returns()


my_stocks_daily_returns       

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights : Using Functions to allocate equal weights to stocks in portfolio
def equal_weights():
    allocation = []
    for stock in stocks_list:
        weight = 1 / len(stocks_list)
        allocation.append(weight)
    return allocation    

weights = equal_weights()

weights

# Calculate portfolio return

my_portfolio_daily_returns = pd.DataFrame(my_stocks_daily_returns.dot(weights))

# Display sample data

my_portfolio_daily_returns.rename(columns = {0:'MY PORTFOLIO'}, inplace = True)

my_portfolio_daily_returns


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame

new_combined_daily_returns = pd.concat([combined_daily_returns, my_portfolio_daily_returns], axis = 'columns', join ='outer')


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
new_combined_daily_returns.dropna(inplace=True)

new_combined_daily_returns

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`

new_combined_annual_std = (new_combined_daily_returns.std())*(np.sqrt(252))

new_combined_annual_std


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

new_combined_rolling_std = new_combined_daily_returns.rolling(window = 21).std()

# Plot rolling standard deviation

new_combined_rolling_std.plot(figsize=(15,10))

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation

new_combined_portfolios_corr = new_combined_daily_returns.corr()

new_combined_portfolios_corr

### Calculate and Plot the 60-day Rolling Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta

my_portfolio_rolling_cov = new_combined_daily_returns['MY PORTFOLIO'].rolling(window=60).cov(new_combined_daily_returns['S&P500'])

sp500_rolling_var = new_combined_daily_returns['S&P500'].rolling(window=60).var()

my_portfolio_rolling_beta = my_portfolio_rolling_cov / sp500_rolling_var

my_portfolio_rolling_beta.plot(figsize=(10,5), title = 'My Portfolio 60-Day Rolling Beta compared to S&P 500')
                                                                                     
                                                                                     

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios

new_combined_portfolios_sharpe = round((new_combined_daily_returns.mean() * 252) / (new_combined_annual_std),2)

new_combined_portfolios_sharpe


In [ ]:
# Visualize the sharpe ratios as a bar plot

new_combined_portfolios_sharpe.plot(kind = 'bar', figsize = (10,5), title='Sharpe Ratios')

### How does your portfolio do?

Write your answer here!

## Stocks Selected ##

For the purpose of analysis, I have selected a list of 5 stocks: Nvidia Corp, Bank of America Corp, Chevron Corp, Shopify Inc and Open Text Corp. The time frame used for data collection is from Feb 01, 2018 to Dec 31, 2019.

## Portfolio Performance ##

I have created a new portfolio by assigning equal weight to all the five stocks selected and compared the returns and risk with other portfolios provided for analysis.

In the time frame used for analysis, the new portfolio, with sharpe ratio of 0.55, gave a better risk adjusted return compared to the S&P 500 and other whale portfolios. Further, while portfolio constructed by Algo 1 was a clear winner, the new portfolio has performed better than Algo 2 portfolio in terms of risk adjusted return.
